In [1]:
# Importing dependencies
import torch
import os
from torch import nn,save,load
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <CAF361F5-1CAC-3EBE-9FC4-4B823D275CAA> /opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/lib/python3.8/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/lib/python3.8/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/computer_vision/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be somet

In [26]:
# Loading Data
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root="data", download=True, train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

TypeError: pic should be PIL Image or ndarray. Got <class 'torch.Tensor'>

In [20]:
# Define the image classifier model
class ImageClassifier(nn.Module):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 22 * 22, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [21]:
# Create an instance of the image classifier model
device = torch.device("cpu")
classifier = ImageClassifier().to(device)

In [22]:
# Define the optimizer and loss function
optimizer = Adam(classifier.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [23]:
# Train the model
for epoch in range(15):  # Train for 10 epochs
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()  # Reset gradients
        outputs = classifier(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    print(f"Epoch:{epoch} loss is {loss.item()}")

Epoch:0 loss is 0.14759425818920135
Epoch:1 loss is 0.061383992433547974
Epoch:2 loss is 0.08135715126991272
Epoch:3 loss is 0.008377352729439735
Epoch:4 loss is 0.22795957326889038
Epoch:5 loss is 0.05627494305372238
Epoch:6 loss is 0.19399894773960114
Epoch:7 loss is 0.0079160425812006
Epoch:8 loss is 0.07223650813102722
Epoch:9 loss is 0.08674051612615585
Epoch:10 loss is 0.09272406250238419
Epoch:11 loss is 0.009752023965120316
Epoch:12 loss is 0.08883432298898697
Epoch:13 loss is 0.0026650105137377977
Epoch:14 loss is 0.07873807102441788


In [24]:
# Create the directory if it doesn't exist
os.makedirs('model_torch_implementation', exist_ok=True)

# Save the trained model
torch.save(classifier.state_dict(), 'model_torch_implementation/model_state.pt')

In [25]:
# Load the test dataset with the appropriate transformations
transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))  # Normalize the images to match training preprocessing
])

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Initialize the model and load the saved state_dict
model = ImageClassifier()
model.load_state_dict(torch.load('model_torch_implementation/model_state.pt'))
model.eval()  # Set the model to evaluation mode

# Evaluate the model's accuracy on the test dataset
correct = 0
total = 0

with torch.no_grad():  # No need to compute gradients during evaluation
     for data, target in test_loader:
          outputs = model(data)
          _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
          total += target.size(0)
          correct += (predicted == target).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

/var/folders/xg/v8q0cj8n7rl0hghy6cp497ph0000gn/T/ipykernel_86094/1148351815.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model_torc

Test Accuracy: 96.78%
